Init workspace


In [1]:
!source .venv/bin/activate

In [ ]:
%pip install -qr requirements.txt clearml  # install

import torch
import utils

display = utils.notebook_init()  # checks

Clone Dataset


In [ ]:
!clearml-data get \
--id 63c8e9373cd44d278ebfe693a575387f \
--copy ../datasets/yolo \
--overwrite

Train


In [ ]:
!python train.py \
--project YOLOv5-Pill-Counter/test \
--name train_bs32_ep75 \
--weights "" \
--cfg yolov5s.yaml \
--data yolo.yaml \
--batch-size 32 \
--epochs 75 \
--img 736 \
--cache

In [ ]:
!python train.py \
--project YOLOv5-Pill-Counter/Evolution \
--name train_bs32_ep30 \
--weights "" \
--cfg yolov5s.yaml \
--data yolo.yaml \
--batch-size 32 \
--epochs 30 \
--img 736 \
--cache \
--evolve \
--resume

Export


In [ ]:
!python export.py \
--data yolo.yaml \
--weights YOLOv5-Pill-Counter/test/train_50/weights/best.pt \
--include tfjs \
--imgsz 736

In [8]:
# YOLOv5 TensorFlow.js export
import re
import subprocess
from pathlib import Path

weights_file = Path(
    "/home/czack913/Code/yolov5/YOLOv5-Pill-Counter/test/train_50/weights/best.pt"
)

f = str(weights_file).replace(".pt", "_web_model")  # js dir
f_pb = weights_file.with_suffix(".pb")  # *.pb path
f_json = f"{f}/model.json"  # *.json path
args = [
    "tensorflowjs_converter",
    "--input_format=tf_frozen_model",
    "--output_format=tfjs_graph_model",
    "--weight_shard_size_bytes=60000000",
    "--output_node_names=Identity,Identity_1,Identity_2,Identity_3",
    str(f_pb),
    f,
]
subprocess.run([arg for arg in args if arg], check=True)

json = Path(f_json).read_text()
with open(f_json, "w") as j:  # sort JSON Identity_* in ascending order
    subst = re.sub(
        r'{"outputs": {"Identity.?.?": {"name": "Identity.?.?"}, '
        r'"Identity.?.?": {"name": "Identity.?.?"}, '
        r'"Identity.?.?": {"name": "Identity.?.?"}, '
        r'"Identity.?.?": {"name": "Identity.?.?"}}}',
        r'{"outputs": {"Identity": {"name": "Identity"}, '
        r'"Identity_1": {"name": "Identity_1"}, '
        r'"Identity_2": {"name": "Identity_2"}, '
        r'"Identity_3": {"name": "Identity_3"}}}',
        json,
    )
    j.write(subst)

2024-01-26 22:09:18.361761: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-26 22:09:18.361872: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-26 22:09:18.363432: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-26 22:09:18.372684: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-26 22:09:19.039029: W tensorflow/compiler/tf2

Validate


In [ ]:
!python val.py \
--weights runs/train/exp2/weights/best.pt \
--data yolo.yaml \
--img 736

Detect


In [ ]:
!python detect.py \
--weights runs/train/exp2/weights/best.pt \
--source yolo.yaml \
--img 736